In [1]:
%load_ext autoreload
%autoreload 2
from glob import glob
import os
import sys
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename

In [20]:
paths_feim = glob('../output/feature_importances/20190907*.gz') \
+ glob('../output/feature_importances/20190906*.gz') \
+ glob('../output/feature_importances/20190905*.gz') \
+ glob('../output/feature_importances/20190904*.gz') \
+ glob('../output/feature_importances/20190903*.gz')
list_good_feature = []

for path in paths_feim:
    feim = read_pkl_gzip(path)
    tmp_list = feim[feim['imp_avg']>500].index.tolist()
    for feature in tmp_list:
        if feature.count('emaildomain'):
            if feature in list_good_feature:
                continue
            list_good_feature.append(feature)

In [21]:
sorted(list_good_feature)

['raw__P_emaildomain_anonymous_com_dummie',
 'raw__P_emaildomain_gmail_com_dummie',
 'raw__P_emaildomain_hotmail_com_dummie',
 'raw__P_emaildomain_mail_com_dummie',
 'raw__P_emaildomain_yahoo_com_dummie',
 'raw__R_emaildomain_anonymous_com_dummie',
 'raw__R_emaildomain_gmail_com_dummie',
 'raw__R_emaildomain_hotmail_com_dummie']

In [109]:
import shutil

list_from_dir = ['raw_use' , 'org_use']
list_from_dir += ['raw_trush' , 'org_trush', 'valid_trush']
to_dir = 'valid'
list_not_found = []

for feature in list_good_feature:
    for from_dir in list_from_dir:
        
        path_train_feature = f"../feature/{from_dir}/{feature}_train.gz"
        path_test_feature = f"../feature/{from_dir}/{feature}_test.gz"
        
        if os.path.exists(path_train_feature) and os.path.exists(path_test_feature):
#         if os.path.exists(path_train_feature):
#         if os.path.exists(path_test_feature):
            try:
                shutil.move(path_train_feature, f'../feature/{to_dir}')
                shutil.move(path_test_feature, f'../feature/{to_dir}')
            except FileNotFoundError:
                print(feature)
                list_not_found.append(feature)